# Neighborhood Based Recommender Systems
Here we implement a user-based recommender system on the MovieLens 100k Dataset.

In [12]:
# Importing packages
import numpy as np
import pandas as pd
import math

In [13]:
# Reads in the list of items from .\\ml-100k\\u.item

items_pd = pd.read_table(".\\ml-100k\\u.item", header=None, names=['item id', 'title', 'release date', 'video release date', 'IMDb URL', 'unknown', 'Action', 'Adventure', 'Animation', 'Children\'s', 'Comedy'
    , 'Crime', 'Documentary', 'Drama', 'Fantasy', 'Film-Noir', 'Horror', 'Musical', 'Mystery', 'Romance', 'Sci-Fi', 'Thriller', 'War', 'Western'], delimiter='|', encoding = "ISO-8859-1")
items_pd.head()
ITEMS = items_pd.shape[0]

# Reads in the list of items from .\\ml-100k\\u.user

users_pd = pd.read_table(".\\ml-100k\\u.user", header=None, names=['user id', 'age', 'gender', 'occupation', 'zip code'], delimiter='|', encoding = "ISO-8859-1")
users_pd.head()
USERS = users_pd.shape[0]

## u1 data set
Here we work with the u1 data set.

In [14]:
# Here we read in the ratings in u1.base

u1_base_pd = pd.read_table(".\\ml-100k\\u1.base", header=None, names=['user id', 'item id', 'rating', 'timestamp'])
u1_base_pd.head()

# rows in u1_base_pd hold data in the form <user id, item id, rating, timestamp>
# each row represents a specific rating

,user id,item id,rating,timestamp
0,1,1,5,874965758
1,1,2,3,876893171
2,1,3,4,878542960
3,1,4,3,876893119
4,1,5,3,889751712


In [15]:
# This function converts the ratings read from u1.base into a matrix with users as rows and items as columns

def to_data_pd(input):
    data_array = np.empty((USERS, ITEMS))
    for _, rating in input.iterrows():
        data_array[rating['user id'] - 1][rating['item id'] - 1] = rating['rating']
    output_pd = pd.DataFrame(data_array, index=range(1,USERS+1), columns=range(1,ITEMS+1))
    return output_pd
    

In [16]:
# put u1_base_pd in a usable format

u1_data_pd = to_data_pd(u1_base_pd)
u1_data_pd.head()

,1,2,3,4,5,6,7,8,9,10,...,1673,1674,1675,1676,1677,1678,1679,1680,1681,1682
1,5.0,3.0,4.0,3.0,3.0,0.0,4.0,1.0,5.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [17]:
# This function finds the average rating for each user

def find_average(data_pd):
    avgs = []
    for _, user in data_pd.iterrows():
        avgs.append(user.sum() / user.astype(bool).sum())

    return avgs

In [18]:
# find each user's average rating

averages = find_average(u1_data_pd)

# find which items each user rated

rated_items = [{k for k in range(1,ITEMS+1) if u1_data_pd[k][i] > 0} for i in range(1,USERS+1)]

Now that most of the prep work is done, it's time to calculate the matrix of Pearson correlation values.

In [19]:
#This implements the Pearson metric as found in equation (2.2)
def pearson(user1, user2):
    intersection = rated_items[user1 - 1].intersection(rated_items[user2 - 1])
    if len(intersection) <= 1:
        return 0

    bottom1 = math.sqrt( sum([(u1_data_pd.iloc[user1-1][k] - averages[user1-1]) ** 2 for k in intersection]))
    bottom2 = math.sqrt( sum([(u1_data_pd.iloc[user2-1][k] - averages[user2-1]) ** 2 for k in intersection]))

    if bottom1 * bottom2 == 0:
        return 0
        
    top = sum([(u1_data_pd.iloc[user1-1][k] - averages[user1-1]) * (u1_data_pd.iloc[user2-1][k] - averages[user2-1]) for k in intersection])

    return top / (bottom1 * bottom2)

In [20]:
# I don't know if this is the way to go about this.
# It takes a long time to run. However, I think it'set
# necessary because we'll have to calculate these values
# sooner or later anyways.

# pearson_matrix = np.empty((USERS,USERS))
# for i in range(1, USERS+1):
#     print(i)
#     # since the matrix is symmetric, we can save time by only calculating half the values
#     for j in range(i, USERS+1):
#         if i == j:
#             pearson_matrix[i - 1][j - 1] = 1
#         pm = pearson(i,j)
#         pearson_matrix[i - 1][j - 1] = pm
#         pearson_matrix[j - 1][i - 1] = pm

In [21]:
# save the results in a csv so we don't have to ever run it again

# u1_pearson_df = pd.DataFrame(pearson_matrix, columns=[i for i in range(1,USERS+1)])
# u1_pearson_df.index = np.arange(1, USERS + 1)
# u1_pearson_df.to_csv(r'.\u1_pearsonmatrix.csv')

In [22]:
# okay let's try to multithread this

# import threading

# THREADCOUNT = 4

# def pearson_matrix_threaded(outputs, i_range):
#     output_np = np.zeros((USERS,USERS))
#     for i in i_range:
#         for j in range(1, USERS):
#             pm = pearson(i,j)
#             print(i, j, pm)
#             output_np[i - 1][j - 1] = pm
#     outputs.append(pd.DataFrame(output_np))

# outputs = []
# threads = [threading.Thread(target=pearson_matrix_threaded, args=(outputs, range(k, USERS, THREADCOUNT),)) for k in range(THREADCOUNT)]
# for thread in threads:
#     thread.start()

# for thread in threads:
#     thread.join()


# df = pd.concat(outputs)
# df.head()

# df.to_csv("pearsonmatrix.csv")




In [23]:
u1_pearson_df = pd.read_csv(r".\u1_pearsonmatrix.csv", usecols=[i for i in range(1,USERS+1)], header=0)
u1_pearson_df.index = np.arange(1, USERS + 1)
u1_pearson_df.head()

,1,2,3,4,5,6,7,8,9,10,...,934,935,936,937,938,939,940,941,942,943
1,1.000000,0.521808,0.525129,1.000000,0.147819,0.298258,0.227549,0.874981,0.907111,-0.044601,...,0.053592,-0.438547,0.486317,-0.181461,0.554811,0.154184,0.250645,0.813325,-0.177845,0.296437
2,0.521808,1.000000,0.140028,-0.514496,0.000000,0.385681,0.775999,0.000000,0.264764,-0.301511,...,0.049314,-0.282424,0.307308,0.644691,0.209520,0.457652,-0.080313,0.032796,0.249705,0.712488
3,0.525129,0.140028,1.000000,0.000000,0.000000,-0.655786,-1.000000,0.435194,0.000000,0.756978,...,0.000000,0.000000,-0.176824,0.356291,-0.244086,-0.424907,-0.108180,0.739940,0.513321,0.000000
4,1.000000,-0.514496,0.000000,1.000000,0.000000,0.000000,-0.497343,0.978363,0.000000,0.000000,...,0.000000,0.000000,-0.210754,-0.256307,0.143209,0.000000,0.839260,0.921244,0.358067,-0.863779
5,0.147819,0.000000,0.000000,0.000000,1.000000,-0.034966,0.249292,0.491733,-0.290883,-0.216527,...,0.323622,-0.622744,-0.224290,0.951357,0.562100,0.913966,0.224538,0.754131,0.249817,0.335129


In [24]:
# Get the set of users that rated each item

rated_users = [{k for k in range(1,USERS+1) if u1_data_pd[i][k] > 0} for i in range(1,ITEMS+1)]

In [26]:
# Of the users who rated item, return the k with highest pearson correlation value with a specified user

def k_closest(user, item, k):
    # pearson_sorted = sorted([u1_pearson_df.iloc[user - 1][i] for i in range(1, USERS+1) if i in rated_users[item - 1]])
    pearson_sorted = sorted((i for i in range(1, USERS+1) if i in rated_users[item - 1]), key=lambda x: u1_pearson_df[str(user)][x])[-k:]
    return pearson_sorted

In [27]:
# Implement r-hat as in equation 2.4

def r_hat(user, item):
    K = k_closest(user, item, 5)
    top = sum(u1_pearson_df[str(user)][user2] * u1_data_pd[item][user2] - averages[user2-1] for user2 in K)
    bot = sum(abs(u1_pearson_df[str(user)][user2]) for user2 in K)

    return averages[user-1] + top/bot

In [28]:
# Here we read in the ratings in u1.test

u1_test_pd = pd.read_table(".\\ml-100k\\u1.test", header=None, names=['user id', 'item id', 'rating', 'timestamp'])
u1_test_pd.head()

# rows in u1_test_pd hold data in the form <user id, item id, rating, timestamp>
# each row represents a specific rating

,user id,item id,rating,timestamp
0,1,6,5,887431973
1,1,10,3,875693118
2,1,12,5,878542960
3,1,14,5,874965706
4,1,17,3,875073198


In [29]:
u1_test_data_pd = to_data_pd(u1_test_pd)
u1_test_data_pd.head()

,1,2,3,4,5,6,7,8,9,10,...,1673,1674,1675,1676,1677,1678,1679,1680,1681,1682
1,0.0,0.0,0.0,0.0,0.0,5.0,0.0,0.0,0.0,3.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,4.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [49]:
u1_pred = np.empty((USERS,ITEMS))
for item in u1_test_data_pd:
    for user in u1_test_data_pd[item].values.nonzero():
        u1_pred[int(user)][item-1] = r_hat(user + 1, item)
u1_pred_df = pd.DataFrame(u1_pred)


KeyError: '[  5  10  15  17  18  20  23  38  41  43  45  49  56  64  66  67  83  94\n  95  96 106 109 117 125 128 131 141 145 148 150 157 158 160 177 181 189\n 194 203 204 213 222 223 232 234 246 247 250 268 271 274 279 280 286 287\n 289 301 308 311 320 322 339 340 345 348 350 359 374 380 396]'